In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np

In [2]:
all_data = pd.read_csv('../data/dataset.csv', encoding="windows-1252", sep=",")

In [302]:
print(all_data.dtypes)

customer           object
product            object
week                int64
forecasted week     int64
demand              int64
dtype: object


In [301]:
pd.DataFrame(all_data)

,customer,product,week,forecasted week,demand
0,DreamLand,Dinosaur,0,0,83
1,DreamLand,Dinosaur,0,1,86
2,DreamLand,Dinosaur,0,2,89
3,DreamLand,Dinosaur,0,3,67
4,DreamLand,Dinosaur,0,4,68
...,...,...,...,...,...
20327,Fun,Willy Wonka,205,206,115
20328,Fun,Willy Wonka,205,207,112
20329,Fun,Willy Wonka,206,206,116
20330,Fun,Willy Wonka,206,207,112


### overview actuals

In [7]:
overview_actuals = all_data[all_data['week'] == all_data['forecasted week']]

In [15]:
with pd.ExcelWriter('../output/overview actuals_GVDK.xlsx') as writer:  
    all_data[all_data['week'] == all_data['forecasted week']].to_excel(writer, sheet_name='Actual Demand', index = False)

In [31]:
overview_forecast_dreamland = all_data[(all_data['week'] <= 25) &  (all_data['customer']=='Dreamland')]

In [43]:
dreamland_dinosaur_tab = all_data[ (all_data['week'] <= 25) &  (all_data['customer']=='DreamLand')&  (all_data['product']=='Dinosaur')]

### overview forecast

In [308]:
print(type(all_data['customer']))

<class 'pandas.core.series.Series'>


In [304]:
print(type(all_data['product']))

<class 'pandas.core.series.Series'>


In [307]:
customers = all_data.customer.unique()

In [305]:
products=all_data['product'].unique()

In [142]:
with pd.ExcelWriter('../output/overview forecasts_GVDK.xlsx') as writer:  
    for customer in customers:
        for product in products:
        #print(customer, product)
            pivot_data = all_data[ (all_data['week'] <= 25)
            &  (all_data['customer']==customer)
            &  (all_data['product']==product)].pivot(
            index='forecasted week', columns='week', values='demand').T
            pivot_data_sheet_name = customer+' - ' +product
            pivot_data.to_excel(writer, sheet_name=pivot_data_sheet_name)
        #display(pivot_data.head())
    

In [46]:
pivot_dreamland_dinosaur_tab = dreamland_dinosaur_tab.pivot(index='forecasted week', columns='week', values='demand').T
with pd.ExcelWriter('../output/overview forecasts_GVDK_new.xlsx') as writer:  
    pivot_dreamland_dinosaur_tab.to_excel(writer, sheet_name='dreamland-dinosaur')

In [108]:
grouped_data = (all_data.groupby(['customer','product','week','forecasted week']))

In [181]:
overview_forecast_dreamland = all_data[(all_data['product'] == 'Dinosaur')
&  (all_data['customer']=='DreamLand')
& (all_data['week'] == all_data['forecasted week'])]

### overview weeks advance

In [348]:
sorted_week = all_data[(all_data['product'] == 'Dinosaur')
&  (all_data['customer']=='DreamLand')
& (all_data['week'] == all_data['forecasted week'])].sort_values('week', ascending=False)

In [349]:
sorted_week = pd.DataFrame(sorted_week[(sorted_week['week'] >= 1)]['demand'])

In [350]:
sorted_week.rename(columns={'demand': '1'}, inplace=True)

In [351]:
sorted_week

,1
5082,103
5080,91
5077,113
5073,108
5068,96
...,...
130,62
104,68
78,68
52,88


In [417]:
created_dataframes = {}
with pd.ExcelWriter('../output/overview_weeks_advance_GVDK.xlsx') as writer:  
    for customer in customers:
        for product in products:
            for weeknr in range(26):
                overview_weeks_advance =all_data[(all_data['product'] == product)
                &  (all_data['customer']==customer)
                & (all_data['week'] == all_data['forecasted week'])].sort_values('week', ascending=False)
                overview_weeks_advance = pd.DataFrame(overview_weeks_advance[(overview_weeks_advance['week'] >= weeknr)]['demand'])
                overview_weeks_advance.rename(columns={'demand': weeknr}, inplace=True)
                created_dataframes[weeknr] =  pd.DataFrame(overview_weeks_advance)
            overview_weeks_advance_sheet_name = customer+' - ' +product
            concatenated_df = pd.concat(created_dataframes.values(), axis=1)
            concatenated_df.to_excel(writer, sheet_name=overview_weeks_advance_sheet_name, index =False)

In [444]:
    created_dataframes = {}
    for customer in customers:
        for product in products:
            for weeknr in range(25):
                stats_weeks_advance =all_data[(all_data['product'] == product)
                &  (all_data['customer']==customer)
                & (all_data['week'] == all_data['forecasted week'])]
                stats_weeks_advance = pd.DataFrame(stats_weeks_advance[(stats_weeks_advance['week'] >= weeknr)]['demand']).describe().T
                stats_weeks_advance.rename(index={'demand': weeknr}, inplace=True)
                stats_weeks_advance = stats_weeks_advance.rename_axis("weeks_in_advance").reset_index()
                created_dataframes[weeknr] =  pd.DataFrame(stats_weeks_advance)
            stats_weeks_advance_sheet_name = customer+' - ' +product
            print(overview_weeks_advance_sheet_name)

Fun - Willy Wonka
Fun - Willy Wonka
Fun - Willy Wonka
Fun - Willy Wonka


### stats weeks advance

In [446]:
created_dataframes = {}
with pd.ExcelWriter('../output/stats_weeks_advance_GVDK.xlsx') as writer: 
    for customer in customers:
        for product in products:
            for weeknr in range(26):
                stats_weeks_advance =all_data[(all_data['product'] == product)
                &  (all_data['customer']==customer)
                & (all_data['week'] == all_data['forecasted week'])]
                stats_weeks_advance = pd.DataFrame(stats_weeks_advance[(stats_weeks_advance['week'] >= weeknr)]['demand']).describe().T
                stats_weeks_advance.rename(index={'demand': weeknr}, inplace=True)
                stats_weeks_advance = stats_weeks_advance.rename_axis("weeks_in_advance").reset_index()
                created_dataframes[weeknr] =  pd.DataFrame(stats_weeks_advance)
            stats_weeks_advance_sheet_name = customer+' - ' +product
            concatenated_df = pd.concat(created_dataframes.values(), axis=0)
            concatenated_df.to_excel(writer, sheet_name=stats_weeks_advance_sheet_name, index =False)

In [447]:
stats_weeks_advance.head()

,weeks_in_advance,count,mean,std,min,25%,50%,75%,max
0,25,183.0,129.36612,15.475338,95.0,118.0,127.0,142.0,173.0


In [405]:
concatenated_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
20331,112,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,...,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0,112.0
20329,116,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,...,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0
20326,114,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,...,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0
20322,114,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,...,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0,114.0
20317,123,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,...,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15353,141,141.0,141.0,141.0,141.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15327,147,147.0,147.0,147.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15301,154,154.0,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15275,120,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [393]:
concatenated_df = pd.concat([df for df in created_dataframes], axis=1)

TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [390]:
pd.DataFrame(created_dataframes, index=[1])

ValueError: Data must be 1-dimensional, got ndarray of shape (208, 1) instead

In [268]:
sorted_week

,demand
5082,103
5080,91
5077,113
5073,108
5068,96
...,...
130,62
104,68
78,68
52,88


In [280]:
sorted_week

,1
5082,103
5080,91
5077,113
5073,108
5068,96
...,...
130,62
104,68
78,68
52,88


In [194]:
overview_forecast_dreamland['demand'](overview_forecast_dreamland.sort_values('week', ascending=False))

TypeError: 'DataFrame' object is not callable

In [198]:
overview_forecast_dreamland.sort_values('week', ascending=False)['demand']

5082    103
5080     91
5077    113
5073    108
5068     96
       ... 
104      68
78       68
52       88
26       85
0        83
Name: demand, Length: 208, dtype: int64

In [341]:
data_1 = {
    'A': ['A1', 'A2', 'A3', 'A4', 'A5']}
df_1 = pd.DataFrame(data_1)

In [342]:
data_2 = {
    'B': ['B1', 'B2', 'B3', 'B4', 'B5']}
df_2 = pd.DataFrame(data_2)

In [343]:
data_3 = {
    'C': ['C1', 'C2', 'C3', 'C4', 'C5']}
df_3 = pd.DataFrame(data_3)

In [345]:
dataframes = [df_1, df_2, df_3]

# Concatenate DataFrames along columns
concatenated_df = pd.concat([df for df in dataframes], axis=1)

# Display the result
print(concatenated_df)

    A   B   C
0  A1  B1  C1
1  A2  B2  C2
2  A3  B3  C3
3  A4  B4  C4
4  A5  B5  C5


In [300]:
# create a dictionary with five fields each
data = {
    'A': ['A1', 'A2', 'A3', 'A4', 'A5'],
    'B': ['B1', 'B2', 'B3', 'B4', 'B4'],
    'C': ['C1', 'C2', 'C3', 'C3', 'C3'],
    'D': ['D1', 'D2', 'D2', 'D2', 'D2'],
    'E': ['E1', 'E1', 'E1', 'E1', 'E1']}
 
# Convert the dictionary into DataFrame
df = pd.DataFrame(data)
 
# Get the unique values of 'B' column
df.B.unique()


array(['B1', 'B2', 'B3', 'B4'], dtype=object)